In [ ]:
%pip install -e ..[dev]

# 🤖 Foundational Models Cloud inference 

In [ ]:
from pprint import pprint
import os
from focoos import DeploymentMode, Focoos, FocoosEnvHostUrl
from dotenv import load_dotenv


load_dotenv()

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

model = focoos.get_model("focoos_object365")
model.deploy(deployment_mode=DeploymentMode.REMOTE, wait=True)
for i in range(500):
    detections = model.infer("./assets/ade_val_034.jpg", threshold=0.4)
    # pprint(detections)

### Annotated preview


In [ ]:
import supervision as sv
import cv2
import numpy as np
import time

preview = model.infer_preview("./assets/ade_val_034.jpg", threshold=0.6)
preview = cv2.imdecode(np.frombuffer(preview, np.uint8), cv2.IMREAD_COLOR)
sv.plot_image(preview, size=(6, 6))

### Unload model from cloud endpoint

In [ ]:
model.unload()

## User Models

In [ ]:
from pprint import pprint

models = focoos.list_models()
pprint(models)

In [ ]:
model = focoos.get_model("c2a157fd006f4a65")
model_info = model.info()
pprint(model_info.status)

# Projects

## List Projects

In [ ]:
from focoos import Focoos, FocoosEnvHostUrl


focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)
pprint(focoos.list_projects())

## Create Project


In [ ]:
from focoos.ports import NewModel
from pprint import pprint
from focoos import Focoos, FocoosEnvHostUrl, Hyperparameters
import os


focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)
project = focoos.create_project(
    project_name="football-players-detection",
    description="football-players-detection",
    dataset_url="s3://focoos-shared-datasets/football-players-detection.v12i.coco.zip",
    dataset_name="football",
    dataset_layout="roboflow_coco",
    task="detection",
)

project = focoos.get_project("7c58c97dbbd54d61")
print(project.metadata)

## Add Model to Project

In [ ]:
from focoos.ports import NewModel
from pprint import pprint
from focoos import Focoos, FocoosEnvHostUrl, Hyperparameters
import os


focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)
project = focoos.get_project("82e2ed94e4b84369")
project.add_model(
    NewModel(
        name="test_01",
        focoos_model="focoos_rtdetr",
        description="Test Model",
        hyperparameters=Hyperparameters(
            batch_size=8, learning_rate=0.001, max_iters=500
        ),
    )
)

model = focoos.get_model("3fb1617977134edb")
project.list_models()
# model.deploy()

## Train a Model

In [ ]:
model = focoos.get_model("3fb1617977134edb")
model.train()